In [3]:
from ttkbootstrap import ttk

In [11]:
import tkinter as tk
import ttkbootstrap as ttk
import time
import backend
#import serial

import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


class AppPanel():

    back = backend.Backend()
    ## Dataset para hacer store de todos los datos
    
    def __init__(self,master):
        self.master = master
        self.master.title("SchrodingerSat")

        self.frame = tk.Frame(self.master, padx= 20, pady=20)
        self.frame.grid(row=0,column=0,sticky="nsew")
        

        self.controlPanel = tk.Frame(self.frame, highlightbackground="blue", highlightthickness=2)
        self.controlPanel.grid(row=0,column = 0,sticky="nsew")

        self.connectionPanel = tk.Frame(self.controlPanel)
        self.connectionPanel.grid(row=0,column=0,sticky="nsew")
        self.connectionPanelName = tk.Label(self.connectionPanel, text="Serial Connection Panel")
        self.connectionPanelName.grid(row=0,column=0,sticky="nsew")
        self.comPortSelector = ttk.Combobox(self.connectionPanel,state="readonly", values=self.back.availablePorts)
        self.comPortSelector.current(4)
        self.comPortSelector.grid(row=1,column=0,sticky="nsew")
        self.comBaudSelector = ttk.Combobox(self.connectionPanel, state="readonly",values=self.back.availableBauds)
        self.comBaudSelector.current(0)
        self.comBaudSelector.grid(row=2,column=0,sticky="nsew")
        self.comButtonPanel = tk.Frame(self.connectionPanel)
        self.comButtonPanel.grid(row=3,column=0,sticky="nsew")
        self.connectButton = tk.Button(self.comButtonPanel, text="Connect",command=self.connect, fg="black", state = tk.NORMAL)
        self.connectButton.grid(row=0,column=0,sticky="nsew")
        self.disconnectButton = tk.Button(self.comButtonPanel, text="Disconnect",command=self.disconnect, fg="black", state=tk.DISABLED)
        self.disconnectButton.grid(row=0,column=1,sticky="nsew")
        self.stateButton = tk

        # Variable de control para el estado del botón
        self.button_state = tk.BooleanVar(value=False)
        # Botón que cambia de color
        self.toggle_button = tk.Button(self.frame, text="Toggle", command=self.toggle_button_color)
        self.toggle_button.grid(row=0, column=0, padx=10, pady=10)
        # Enlace del botón con la función de cambio de color
        self.toggle_button.bind("<Button-1>", self.toggle_button_color)

        self.trialPanel = tk.Frame(self.controlPanel)
        self.trialPanel.grid(row=2, column= 0,sticky="nsew")
        self.trialPanelName = tk.Label(self.trialPanel, text="Trial Panel")
        self.trialPanelName.grid(row=0,column=0,sticky="nsew")
        self.startStopFrame = tk.Frame(self.trialPanel)
        self.startStopFrame.grid(row=1,column=0,sticky="nsew")
        self.startButton = tk.Button(self.startStopFrame, text="Start", command=self.startReading, fg="black", state = tk.DISABLED)
        self.startButton.grid(row=0,column=0,sticky="nsew")
        self.stopButton = tk.Button(self.startStopFrame, text="Stop", command=self.stopReading, fg="black",state = tk.DISABLED)
        self.stopButton.grid(row=0,column=1,sticky="nsew")
        self.saveButton = tk.Button(self.trialPanel, text="Save Data",command=self.saveData, fg="black",state = tk.DISABLED)
        self.saveButton.grid(row=2,column=0,sticky="nsew")

        self.motionVarPanel = tk.Frame(self.frame,padx=5,pady=5, highlightbackground="blue", highlightthickness=2)
        self.motionVarPanel.grid(row=0,column=1,sticky="nsew")
        self.motionVarPanelName = tk.Label(self.motionVarPanel, text="Motion Variables")
        self.motionVarPanelName.grid(row=0,column=0,sticky="nsew")
        self.plotFrame = tk.Frame(self.motionVarPanel,highlightbackground="red", highlightthickness=2)
        self.plotFrame.grid(row=1,column=0,sticky="nsew")
        self.firstFigure, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.firstFigure, master=self.plotFrame)
        self.canvas.get_tk_widget().grid(row=0,column=0,sticky="nsew")
        self.updatePlot()


    def connect(self):
        port = self.comPortSelector.get()
        baud = int(self.comBaudSelector.get()) if type(self.comBaudSelector.get()) != int else self.comBaudSelector.get()

        self.back.serialConnect(port,baud,self.connectButton,self.disconnectButton)

        self.startButton.config(state=tk.NORMAL)
        #self.stopButton.config(state=tk.NORMAL)
        #self.saveButton.config(state=tk.NORMAL)
        
    def disconnect(self):
        self.back.serialDisconnect(self.connectButton,self.disconnectButton)
        self.startButton.config(state=tk.DISABLED)
        self.stopButton.config(state=tk.DISABLED)
        self.saveButton.config(state=tk.DISABLED)

    def startReading(self):
        self.back.readActive = True
        print(f"Serial lecture started")
        self.back.initDatabase()
        self.stopButton.config(state=tk.NORMAL)
        self.startButton.config(state=tk.DISABLED)
        self.master.after(10, self.readAndWrite)


    def readAndWrite(self):
        self.back.serialReadline()
        self.back.updateDB()
        self.master.after(20, self.readAndWrite)

    
    def stopReading(self):
        self.back.readActive = False
        self.back.dataBuffer = None
        print(f"Serial lecture stopped")
        self.startButton.config(state=tk.NORMAL)
        self.stopButton.config(state=tk.DISABLED)
        self.saveButton.config(state=tk.NORMAL)

    def saveData(self):
        self.back.saveData()
        self.saveButton.config(state=tk.DISABLED)
    
    def updatePlot(self):

        if self.back.db is not None:
            x = self.back.db.index.to_list()
            y = self.back.db.loc[:,"time"]

            self.ax.clear()
            self.ax.plot(x, y)
            self.ax.set_xlabel('X-axis')
            self.ax.set_ylabel('Y-axis')
            self.ax.set_title('Time Plot')

            self.canvas.draw()

        self.master.after(20, self.updatePlot)

def main():
    root = tk.Tk()
    root.geometry("1920x1080")
    app = AppPanel(root)
    root.resizable(True,True)
    root.mainloop()

if __name__ == "__main__":
    main()

AttributeError: 'AppPanel' object has no attribute 'toggle_button_color'